In [2]:
%pip install peft

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple
     ---------------------------------------- 0.0/411.1 kB ? eta -:--:--
      --------------------------------------- 10.2/411.1 kB ? eta -:--:--
      --------------------------------------- 10.2/411.1 kB ? eta -:--:--
     -- ---------------------------------- 30.7/411.1 kB 186.2 kB/s eta 0:00:03
     -- ---------------------------------- 30.7/411.1 kB 186.2 kB/s eta 0:00:03
     -- ---------------------------------- 30.7/411.1 kB 186.2 kB/s eta 0:00:03
     -- ---------------------------------- 30.7/411.1 kB 186.2 kB/s eta 0:00:03
     -- ---------------------------------- 30.7/411.1 kB 186.2 kB/s eta 0:00:03
     -- ---------------------------------- 30.7/411.1 kB 186.2 kB/s eta 0:00:03
     -- ---------------------------------- 30.7/411.1 kB 186.2 kB/s eta 0:00:03
     ------ ------------------------------ 71.7/411.1 kB 140.3 kB/s eta 0:00:03
     ------ ------------------------------ 71.7/411.1 kB 140.3 kB/s eta 0:00:03

In [3]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from datasets import load_dataset, Dataset  # Dataset用于从列表创建数据集

# --- 0. 配置参数 ---
# 替换为你要使用的基础模型路径或Hugging Face Hub ID
# 常用模型示例: "THUDM/chatglm2-6b", "baichuan-inc/Baichuan2-7B-Base", "mistralai/Mistral-7B-v0.1", "gpt2"
model_name_or_path = "gpt2"  # 示例使用GPT-2，你可以替换为更大的模型

output_dir = "./lora_finetuned_model_gpt2"  # 模型保存路径

# LoRA配置
lora_r = 8  # LoRA的秩，影响适配器参数量和表达能力，越大参数越多
lora_alpha = 16  # LoRA的缩放因子，通常设为 r 的2倍
lora_dropout = 0.05  # LoRA层的Dropout率
# target_modules: 关键！根据你使用的模型类型调整
# 对于 Llama/Mistral/Baichuan 等: ["q_proj", "k_proj", "v_proj"] 是常见选择
# 对于 ChatGLM2/3: ["query_key_value"]
# 对于 GPT-2: ["c_attn"] (或者更全面地，可以包含 c_proj, mlp.c_fc, mlp.c_proj等)
# 推荐使用 peft.tuners.lora.model.find_all_linear_names 来自动查找线性层
# 或者查看模型结构 model.print_trainable_parameters() (运行到 get_peft_model 之后)
lora_target_modules = ["c_attn"]  # GPT-2 的LoRA目标模块

# 训练参数
training_epochs = 3
learning_rate = 2e-4
per_device_train_batch_size = 4
gradient_accumulation_steps = 1  # 增加这个可以模拟更大的batch size，节省显存
# QLoRA (4比特量化) 配置
use_4bit_quantization = False  # 如果你的GPU显存较大，可以设置为False
# 如果显存不够，强烈建议设置为 True，需要安装 bitsandbytes
# 如果设置为True，优化器通常选 paged_adamw_8bit

# --- 1. 数据准备 ---
# 示例：创建一个简单的指令微调数据集
# 实际应用中，你会从文件加载，例如 load_dataset('json', data_files={'train': 'your_data.jsonl'})
raw_data = [
    {"instruction": "请问中国的首都是哪里？", "output": "中国的首都是北京。"},
    {"instruction": "列举三个水果。", "output": "苹果、香蕉、橙子。"},
    {
        "instruction": "介绍一下LoRA微调技术。",
        "input": "",
        "output": "LoRA（Low-Rank Adaptation）是一种参数高效的微调技术，它通过在预训练模型的每一层注入一小部分可训练的低秩矩阵来更新模型，而无需修改原始模型的权重。",
    },
    {"instruction": "计算1加1等于多少？", "input": "", "output": "1加1等于2。"},
]
dataset = Dataset.from_list(raw_data)


# 定义数据格式化函数（指令微调通常需要特定的prompt格式）
def format_instruction_dataset(example):
    # Alpaca风格指令微调格式
    if "input" in example and example["input"]:
        prompt = f"### Instruction:\n{example['instruction']}\n### Input:\n{example['input']}\n### Output:\n"
    else:
        prompt = f"### Instruction:\n{example['instruction']}\n### Output:\n"

    return {"text": prompt + example["output"]}


# 应用格式化函数到数据集
dataset = dataset.map(format_instruction_dataset)

# --- 2. 加载基础模型和分词器 ---
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
# 检查并设置pad_token，某些模型可能没有
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print(
        f"Warning: tokenizer.pad_token was None, set to tokenizer.eos_token: {tokenizer.pad_token}"
    )

# 根据是否使用4比特量化加载模型
if use_4bit_quantization:
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        load_in_4bit=True,  # 启用4比特量化
        torch_dtype=torch.bfloat16,  # 计算数据类型，bfloat16在Ampere及更新架构上性能好
        device_map="auto",  # 自动分配到GPU
        trust_remote_code=True,
    )
    # 针对量化模型，PEFT 推荐使用 prepare_model_for_kbit_training
    # 它会做一些处理，如设置梯度检查点、调整LayerNorm的dtype等
    model = prepare_model_for_kbit_training(model)
else:
    model = AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        torch_dtype=torch.float16,  # 或 torch.bfloat16，根据GPU能力选择
        device_map="auto",
        trust_remote_code=True,
    )

model.config.use_cache = False  # 训练时禁用cache，可以节省显存

# --- 3. 配置 LoRA (LoraConfig) ---
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,  # 任务类型：因果语言模型 (文本生成)
    inference_mode=False,  # 训练模式
    r=lora_r,  # LoRA秩
    lora_alpha=lora_alpha,  # LoRA缩放因子
    lora_dropout=lora_dropout,  # LoRA dropout
    target_modules=lora_target_modules,  # 目标模块
    bias="none",  # 偏置项通常不参与微调
)

# --- 4. 准备 PEFT 模型 ---
# 将基础模型和LoRA配置结合，生成PEFT模型
model = get_peft_model(model, peft_config)

# 打印模型可训练参数量，你会发现它非常小！
model.print_trainable_parameters()
# 示例输出: trainable params: 393216 || all params: 124430400 || trainable%: 0.31601768832626075


# --- 5. 数据 Tokenization ---
def tokenize_function(examples):
    # 对于Causal LM，我们通常拼接所有文本并设置 max_length
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,  # 根据模型和数据调整最大长度
        padding="max_length",  # 对齐到最大长度
    )


tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=[
        "text"
    ],  # 移除原始文本列，只保留 tokenized 后的 input_ids, attention_mask
)

# --- 6. 设置训练器 (Trainer) ---
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=training_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    logging_dir=f"{output_dir}/logs",
    logging_steps=10,
    save_steps=100,  # 每100步保存一次模型
    save_total_limit=3,  # 最多保存3个检查点
    fp16=True if torch.cuda.is_available() else False,  # 启用混合精度训练
    bf16=(
        True if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else False
    ),
    # 优化器选择：对于QLoRA，推荐 paged_adamw_8bit
    optim="paged_adamw_8bit" if use_4bit_quantization else "adamw_torch",
    warmup_steps=100,
    lr_scheduler_type="cosine",
    report_to="tensorboard",  # 可以将训练过程可视化到TensorBoard
)

# Data Collator 负责将 tokenize 后的数据批处理，并进行 padding
# mlm=False 表示非掩码语言模型，适用于因果语言模型
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# --- 7. 开始训练 ---
print("开始LoRA微调...")
trainer.train()
print("LoRA微调完成！")

# --- 8. 保存 LoRA 适配器 ---
# PEFT 库只会保存 LoRA 适配器（很小），而不是整个模型
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)  # 分词器也要保存
print(f"LoRA适配器已保存到: {output_dir}")


# --- 9. 加载 LoRA 适配器进行推理或合并 ---

print("\n--- 加载LoRA适配器进行推理 (推荐方式) ---")
from peft import PeftModel

# 重新加载基础模型 (推理时不需要进行kbit训练，所以可以不加 prepare_model_for_kbit_training)
# 注意：这里的dtype应与基础模型一致，或者使用fp16/bf16
base_model_for_inference = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    torch_dtype=torch.float16,  # 推理时也建议使用fp16以节省显存
    device_map="auto",
    trust_remote_code=True,
)

# 通过 from_pretrained 方法加载 LoRA 适配器到基础模型
peft_model_for_inference = PeftModel.from_pretrained(
    base_model_for_inference, output_dir
)
peft_model_for_inference.eval()  # 设置为评估模式

# 进行推理
prompt_text = "### Instruction:\n请问LoRA微调的优势是什么？\n### Output:\n"
inputs = tokenizer(prompt_text, return_tensors="pt").to("cuda")  # 确保输入在GPU上

with torch.no_grad():
    outputs = peft_model_for_inference.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,  # 采样生成
        top_p=0.9,  # Top-p 采样
        temperature=0.7,  # 温度
        repetition_penalty=1.1,  # 惩罚重复
    )
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"原始prompt:\n{prompt_text}")
print(f"生成结果:\n{generated_text}")


# --- 10. (可选) 将 LoRA 适配器合并回基础模型 ---
# 合并后的模型不再是PEFT模型，而是标准的transformers模型，可以像普通模型一样部署。
# 注意：如果基础模型是4bit量化的，merge_and_unload()会将其卸载为全精度(如fp16)，这将增加模型大小。
print("\n--- 合并LoRA适配器回基础模型 (生成完整模型文件) ---")

# 使用之前加载的peft_model_for_inference进行合并
merged_model = peft_model_for_inference.merge_and_unload()

# 保存合并后的完整模型 (这将保存所有参数，文件会很大，与原始基础模型大小相近)
merged_output_dir = "./merged_finetuned_model_gpt2"
merged_model.save_pretrained(merged_output_dir)
tokenizer.save_pretrained(merged_output_dir)
print(f"合并后的完整模型已保存到: {merged_output_dir}")

# 现在可以像加载普通Transformers模型一样加载 merged_model
# loaded_merged_model = AutoModelForCausalLM.from_pretrained(merged_output_dir, device_map="auto")
# loaded_tokenizer = AutoTokenizer.from_pretrained(merged_output_dir)

Error importing huggingface_hub._snapshot_download: cannot import name 'GatedRepoError' from 'huggingface_hub.errors' (d:\Program\Anaconda\envs\py310\lib\site-packages\huggingface_hub\errors.py)


ImportError: cannot import name 'GatedRepoError' from 'huggingface_hub.errors' (d:\Program\Anaconda\envs\py310\lib\site-packages\huggingface_hub\errors.py)